# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846834301281                   -0.70    4.8         
  2   -7.852325109331       -2.26       -1.53    1.0   25.4ms
  3   -7.852615603433       -3.54       -2.55    1.5   26.2ms
  4   -7.852645935908       -4.52       -2.88    2.5   32.8ms
  5   -7.852646508573       -6.24       -3.20    1.0   25.0ms
  6   -7.852646679021       -6.77       -4.07    1.0   24.8ms
  7   -7.852646686673       -8.12       -5.03    2.0   30.9ms
  8   -7.852646686726      -10.27       -5.50    2.0   29.8ms
  9   -7.852646686728      -11.73       -5.57    1.2   27.6ms
 10   -7.852646686730      -11.69       -6.71    1.0   24.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846851088952                   -0.70           4.8         
  2   -7.852528386353       -2.25       -1.63   0.80    2.0    261ms
  3   -7.852636798830       -3.96       -2.71   0.80    1.0   22.2ms
  4   -7.852646514813       -5.01       -3.31   0.80    2.2   29.3ms
  5   -7.852646682472       -6.78       -4.17   0.80    1.2   23.9ms
  6   -7.852646686569       -8.39       -4.86   0.80    1.5   25.8ms
  7   -7.852646686723       -9.81       -5.79   0.80    1.8   25.8ms
  8   -7.852646686730      -11.16       -6.56   0.80    2.0   27.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.394532e+01     3.720513e+00
 * time: 0.433380126953125
     1     1.179567e+00     1.927170e+00
 * time: 0.6305720806121826
     2    -1.695303e+00     2.185482e+00
 * time: 0.6552009582519531
     3    -3.652797e+00     2.011982e+00
 * time: 0.6907980442047119
     4    -4.980350e+00     1.790057e+00
 * time: 0.7261781692504883
     5    -6.660491e+00     1.206623e+00
 * time: 0.8539531230926514
     6    -7.381854e+00     7.494675e-01
 * time: 0.8891491889953613
     7    -7.578717e+00     7.752291e-01
 * time: 0.9134869575500488
     8    -7.743448e+00     2.944023e-01
 * time: 0.9380171298980713
     9    -7.800016e+00     8.032626e-02
 * time: 0.9626071453094482
    10    -7.824661e+00     6.420450e-02
 * time: 0.9872310161590576
    11    -7.836061e+00     9.565533e-02
 * time: 1.0118730068206787
    12    -7.844577e+00     4.615342e-02
 * time: 1.0364091396331787
    13    -7.847503e+00     4.513701e-02
 * time: 1.06069898605

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846786948753                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645900895                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.49s
  3   -7.852646686730      -13.27       -7.24    149ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.8544369429516827e-7
|ρ_newton - ρ_scfv| = 1.291980279561536e-7
|ρ_newton - ρ_dm|   = 4.544984996665822e-10
